In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
import shap
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
dataset = pd.read_csv('/Users/henryding/Library/CloudStorage/OneDrive-cmu.edu.cn/sci/复杂网络动态模型/database/immunology-variables.csv')
#dataset = dataset.iloc[:,1:50]

#dataset[2,29] = pd.to_numeric(dataset[2,29])
#上面的命令是把这一列转换为float格式
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
from pandas import DataFrame
N1 = dataset.iloc[:,0]
L1 = dataset.iloc[:,1]
M1 = dataset.iloc[:,2]
N2 = dataset.iloc[:,3]
L2 = dataset.iloc[:,4]
M2 = dataset.iloc[:,5]
N3 = dataset.iloc[:,6]
L3 = dataset.iloc[:,7]
M3 = dataset.iloc[:,8]
N4 = dataset.iloc[:,9]
L4 = dataset.iloc[:,10]
M4 = dataset.iloc[:,11]
Target = dataset.iloc[:, 12]
#print(Target)#源数据基础上的Outcome列

a1 = N1/L1; a2 = L1/M1; a3 = N1/M1; a4 = L1/N1; a5 = M1/L1; a6 = M1/N1
b1 = a1 + a5; b2 = a2 + a3; b3 = a4 + a6; b4 = a1 * a5; b5 = a2 * a3; b6 = a4 * a6
c1 = N2/L2; c2 = L2/M2; c3 = N2/M2; c4 = L2/N2; c5 = M2/L2; c6 = M2/N2
d1 = c1 + c5; d2 = c2 + c3; d3 = c4 + c6;d4 = c1 * c5;d5 = c2 * c3;d6 = c4 * c6;
e1 = N3/L3;e2 = L3/M3;e3 = N3/M3;e4 = L3/N3;e5 = M3/L3;e6 = M3/N3;
f1 = e1 + e5;f2 = e2 + e3;f3 = e4 + e6;f4 = e1 * e5;f5 = e2 * e3;f6 = e4 * e6
g1 = N4/L4;g2 = L4/M4;g3 = N4/M4;g4 = L4/N4;g5 = M4/L4;g6 = M4/N4;
h1 = g1 + g5;h2 = g2 + g3;h3 = g4 + g6;h4 = g1 * g5;h5 = g2 * g3;h6 = g4 * g6
a11 = (N2-N1)/N1; a12 = (N3-N2)/N2; a13 = (N4-N3)/N3; a14 = (L2-L1)/L1; a15 = (L3-L2)/L2; a16 = (L4-L3)/L3; a17 = (M2-M1)/M1; a18 = (M3-M2)/M2; a19 = (M4-M3)/M3
b11 = a11*a1; b12 = a12*c1; b13 = a13*e1; b14 = a14*a2; b15 = a15*c2; b16 = a16*e2; b17 = a17*a3; b18 = a18*c3; b19 = a19 * e3
c11 = a11/a12; c12 = a12/a13; c13 = a14/a15; c14 = a15/a16; c15 = a17/a18; c16 = a18/a19



#data = pd.merge(a1,a2,a3 ,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,h1,h2,h3,h4,h5,h6)
data = pd.concat(objs=[Target,a1,a2,a3 ,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,h1,h2,h3,h4,h5,h6,
                       a11,a12,a13,a14,a15,a16,a17,a18,a19,b11,b12,b13,b14,b15,b16,b17,b18,b19,N1,M1,L1,N2,M2,L2,N3,M3,L3,N4,M4,L4],axis=1)
names = ['Outcome','a1','a2','a3','a4','a5','a6','b1','b2','b3','b4','b5','b6','c1','c2','c3','c4','c5','c6','d1','d2','d3','d4','d5','d6','e1','e2','e3','e4','e5','e6','f1','f2','f3','f4','f5','f6','g1','g2','g3','g4','g5','g6','h1','h2','h3','h4','h5','h6',
          'a11','a12','a13','a14','a15','a16','a17','a18','a19','b11','b12','b13','b14','b15','b16','b17','b18','b19','N1','M1','L1','N2','M2','L2','N3','M3','L3','N4','M4','L4']

data.columns = names
data.head()


In [ ]:
rcParams['figure.figsize'] = 40, 80

plt.matshow(data.corr('kendall'), cmap=plt.cm.seismic)
plt.yticks(np.arange(data.shape[1]), data.columns)
plt.xticks(np.arange(data.shape[1]), data.columns, rotation=90)
plt.colorbar()
#plt.show()

In [ ]:
data.hist()

In [ ]:
data["Outcome"].describe()
data["Outcome"].unique()
print(data.corr()["Outcome"].abs().sort_values(ascending=False))

In [ ]:
y = data["Outcome"]

sns.countplot(y)


Target_temp = dataset.Outcome.value_counts()

print(Target_temp)

In [ ]:
from sklearn.model_selection import train_test_split

predictors = data.drop("Outcome",axis=1)
Target = data["Outcome"]

X_train,X_test,Y_train,Y_test = train_test_split(predictors,Target,test_size=0.50,random_state=0)
X_train.shape
X_test.shape
Y_train.shape
Y_test.shape

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(X_train,Y_train)

Y_pred_lr = lr.predict(X_test)
Y_pred_lr.shape
score_lr = round(accuracy_score(Y_pred_lr,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")
# save the model to disk
filename = 'LogisticRegression.sav'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100,2)

print("The accuracy score achieved using Naive Bayes is: "+str(score_nb)+" %")

In [ ]:
from sklearn import svm

sv = svm.SVC(kernel='linear')

sv.fit(X_train, Y_train)

Y_pred_svm = sv.predict(X_test)

Y_pred_svm.shape

score_svm = round(accuracy_score(Y_pred_svm,Y_test)*100,2)

print("The accuracy score achieved using Linear SVM is: "+str(score_svm)+" %")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,Y_train)
Y_pred_knn=knn.predict(X_test)

Y_pred_knn.shape

score_knn = round(accuracy_score(Y_pred_knn,Y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")

In [ ]:
#决策树分类器
from sklearn.tree import DecisionTreeClassifier

max_accuracy = 0


for x in range(200):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train,Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)


dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train,Y_train)
Y_pred_dt = dt.predict(X_test)

print(Y_pred_dt.shape)

score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")

In [ ]:
#随机森林分类器
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(1000):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train,Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict(X_test)

Y_pred_rf.shape

score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_rf)+" %")

In [ ]:
#XGboost分类器——类似stacking原理，合并了多个弱分类器的强分类器

import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
model = xgb_model.fit(X_train, Y_train)

Y_pred_xgb = xgb_model.predict(X_test)
Y_pred_xgb.shape

score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100,2)

print("The accuracy score achieved using XGBoost is: "+str(score_xgb)+" %")

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X_train)
shap.plots.waterfall(shap_values[0])#可视化


In [ ]:
#根据label画图，这一步构建的是3D坐标系
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
fig = plt.figure()
ax = plt.axes(projection='3d')

In [ ]:
#3D坐标系的内容填充
X = data['e2']
Y = data['b3']
Z = data['Outcome']
T = data['a15']
C = []
for z in Z:
    if z >= 1:
        C.append("r")
#   elif z < 1:
#       C.append("y")
    else:
        C.append("k")
#作图
ax=plt.subplot(111,projection='3d')

ax.scatter(X,Y,T,c=C,alpha=0.4,s=200)#生成散点.利用c控制颜色序列,s控制大小

ax.set_xlabel('vactor1')

ax.set_ylabel('vactor2')

ax.set_zlabel('outcome')
plt.show()